In [ ]:
# import necessary libraries

import os
import time
import json
import requests
import logging
import concurrent.futures
from dotenv import load_dotenv
import tenacity  # for retrying failed requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import openpyxl
import pycountry
import random

In [ ]:
# Load environment variables from .env file
load_dotenv()

API_KEY = os.getenv("API_KEY")
if not API_KEY:
    raise ValueError("❌ API_KEY مش موجود في ملف .env")

OUT_DIR = "data"
os.makedirs(OUT_DIR, exist_ok=True)
OUT_FILE = os.path.join(OUT_DIR, "movies_all.ndjson")

logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

In [ ]:
# Function to fetch data from TMDB API with retries and exponential backoff

def fetch(endpoint, params=None, retries=10, backoff=2):
    base_url = "https://api.themoviedb.org/3"
    params = params or {}
    params["api_key"] = API_KEY
    url = f"{base_url}{endpoint}"

    for i in range(retries):
        try:
            response = requests.get(url, params=params, timeout=60)
            response.raise_for_status()
            return response.json()
        except (requests.exceptions.ConnectionError,
                requests.exceptions.Timeout) as e:
            wait = backoff * (i + 1)
            logger.warning(f"⚠️ Network error: {e}. Retrying in {wait}s...")
            time.sleep(wait)
        except Exception as e:
            logger.error(f"❌ Fatal error: {e}")
            raise
    raise Exception("Failed after retries")

In [ ]:
# Function to generate date ranges (yearly or half-yearly)

# step can be "year" or "half"
def generate_date_ranges(start_year=1900, end_year=2025, step="year"):
    ranges = []
    for y in range(start_year, end_year + 1):
        if step == "year":
            start = f"{y}-01-01"
            end = f"{y}-12-31"
            ranges.append((start, end))
        elif step == "half":
            # First half
            ranges.append((f"{y}-01-01", f"{y}-06-30"))
            # Second half
            ranges.append((f"{y}-07-01", f"{y}-12-31"))
    return ranges

In [ ]:
# Function to generate quarter date ranges

def generate_quarter_ranges(start_year=1900, end_year=2025):
    quarters = [
        ("01-01", "03-31"),
        ("04-01", "06-30"),
        ("07-01", "09-30"),
        ("10-01", "12-31"),
    ]
    ranges = []
    for year in range(start_year, end_year + 1):
        for q_start, q_end in quarters:
            start = f"{year}-{q_start}"
            end = f"{year}-{q_end}"
            ranges.append((start, end))
    return ranges

In [ ]:
# Function to generate monthly date ranges

def generate_monthly_ranges(start_year=1900, end_year=2025):
    import calendar
    ranges = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            start = f"{year}-{month:02d}-01"
            last_day = calendar.monthrange(
                year, month)[1]  # Get last day of the month
            end = f"{year}-{month:02d}-{last_day}"
            ranges.append((start, end))
    return ranges

In [ ]:
# Function to discover movies with pagination and optional date range filtering

def discover_movies(pages=5, date_range=None, sort_by="popularity.desc"):
    results = []
    params = {"sort_by": sort_by}

    if date_range:
        params["primary_release_date.gte"] = date_range[0]
        params["primary_release_date.lte"] = date_range[1]

    for p in range(1, pages + 1):
        logger.info(f"--- Discover page {p} --- (range={date_range})")
        params["page"] = p
        r = fetch("/discover/movie", params)
        results.extend(r.get("results", []))

        # stop early if last page
        if p >= r.get("total_pages", 1):
            break

        time.sleep(0.1)  # respect rate-limit
    return results

In [ ]:
# Function to process a single movie and append details to ndjson file
def process_movie(movie_id, out_file):
    try:
        data = fetch(f"/movie/{movie_id}",
                     # Fetch movie details with credits and reviews
                     {"append_to_response": "credits,reviews"})
        with open(out_file, "a", encoding="utf-8") as f:
            # Append movie data to ndjson file
            f.write(json.dumps(data, ensure_ascii=False) + "\n")

        return None
    except Exception as e:
        logger.error(f"Failed {movie_id}: {e}")
        return movie_id

In [ ]:
# Function to process list of IDs concurrently

def process_ids(movie_ids, out_file, workers=16):  # number of concurrent workers to use
    failed = []
    # Use ThreadPoolExecutor for I/O bound tasks
    with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
        futures = [executor.submit(process_movie, mid, out_file)  # Submit tasks to executor
                   for mid in movie_ids]
        # Process completed futures
        for fut in concurrent.futures.as_completed(futures):
            res = fut.result()
            if res:
                failed.append(res)
    return failed

In [ ]:
# Function to load existing IDs from ndjson file to avoid duplicates
def load_existing_ids(out_file):
    existing_ids = set()
    if os.path.exists(out_file):
        with open(out_file, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    data = json.loads(line)
                    if "id" in data:
                        existing_ids.add(data["id"])
                except json.JSONDecodeError:
                    continue
    return existing_ids

In [ ]:

# Main pipeline function

def main(pages=500, workers=16):
    all_movies = []
    seen = load_existing_ids(OUT_FILE)

    logger.info(f"Resume mode: Found {len(seen)} movies already saved ✅")

    # Read values from .env
    step = os.getenv("DATE_STEP", "year")  # "year" or "half"
    start_year = int(os.getenv("START_YEAR", "1900"))
    end_year = int(os.getenv("END_YEAR", "2025"))

    # ranges = generate_date_ranges(start_year, end_year, step=step)
    # ranges = generate_quarter_ranges(start_year, end_year)
    ranges = generate_monthly_ranges(start_year, end_year)

    for start, end in ranges:
        logger.info(f"Fetching movies between {start} and {end}")
        movies = discover_movies(pages=pages, date_range=(start, end))
        ids = [m["id"] for m in movies if m["id"] not in seen]
        seen.update(ids)
        all_movies.extend(ids)
        logger.info(f"Collected {len(ids)} new movies for {start} → {end}")

        # Save all into one file
        failed_ids = process_ids(ids, OUT_FILE, workers)

        # Retry failed IDs
        if failed_ids:
            logger.info(f"Retrying {len(failed_ids)} failed movies...")
            retry_failed = process_ids(failed_ids, OUT_FILE, workers)
            if retry_failed:
                fail_file = os.path.join(OUT_DIR, "failed_ids.txt")
                with open(fail_file, "w") as f:
                    f.write("\n".join(map(str, retry_failed)))
                logger.info(
                    f"Still failed after retry. Saved {len(retry_failed)} IDs to {fail_file}")
            else:
                logger.info("All failed IDs succeeded on retry ✅")

    logger.info(f"Total new movies downloaded: {len(all_movies)}")
    logger.info(f"Final data saved in {OUT_FILE}")
    logger.info("Pipeline finished!")


if __name__ == "__main__":
    main()

In [ ]:
input_file = r"D:\depi_project\data\raw\movies_all.ndjson"   # file Large JSON
output_file = r"D:\depi_project\data\raw\movies_all.csv"  # output CSV file

# write CSV in chunks
chunksize = 100000
batch = []
count = 0

with open(input_file, "r", encoding="utf-8") as f, open(output_file, "w", encoding="utf-8", newline="") as out_csv:
    writer = None

    for line in f:
        try:
            record = json.loads(line.strip())  # convert line to dict
            batch.append(record)
        except json.JSONDecodeError:
            continue  # If a line is corrupted, skip it

        if len(batch) >= chunksize:
            df = pd.DataFrame(batch)

            if writer is None:
                df.to_csv(out_csv, index=False, header=True)
                writer = True
            else:
                df.to_csv(out_csv, index=False, header=False)

            count += len(batch)
            print(f"✅ Processed {count:,} records...")
            batch = []

    # Write remaining records
    if batch:
        df = pd.DataFrame(batch)
        if writer is None:
            df.to_csv(out_csv, index=False, header=True)
        else:
            df.to_csv(out_csv, index=False, header=False)
        count += len(batch)

print(f"🎉 Finished! Total {count:,} records saved to {output_file}")